In [1]:
# Either you can store the  OpenAI key in the “OPENAI_API_KEY” environment variable.
# or pass it here as below from a config.ini
import configparser
workingFolder=r'C:\Users\jfrancis\OneDrive - GalaxE. Solutions, Inc\GalaxE D Drive\AI Journey\Gen AI'
# Read the configuration file
config = configparser.ConfigParser()
config.read(workingFolder+'\\config.ini')
OPENAI_API_KEY=config.get('General','OPENAI_API_KEY')
ACTIVELOOP_TOKEN=config.get('General','ACTIVELOOP_TOKEN')
ACTIVELOOP_ORG_ID=config.get('General','ACTIVELOOP_ORG_ID')
HUGGINGFACEHUB_API_TOKEN=config.get('General','HUGGINGFACEHUB_API_TOKEN')
GOOGLE_API_KEY=config.get('General','GOOGLE_API_KEY')
GOOGLE_CSE_ID=config.get('General','GOOGLE_CSE_ID')

### Example selectors:

Example selectors can be used to provide a few-shot learning experience. The primary goal of few-shot learning is to learn a similarity function that maps the similarities between classes in the support and query sets. In this context, an example selector can be designed to choose a set of relevant examples that are representative of the desired output.

The ExampleSelector is used to select a subset of examples that will be most informative for the language model. This helps in generating a prompt that is more likely to generate a good response. 

In [2]:
# Get the token from OPENAI/Active loop website before this. Now we are taking from the config.ini
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ["ACTIVELOOP_TOKEN"] = ACTIVELOOP_TOKEN
# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = ACTIVELOOP_ORG_ID
my_activeloop_dataset_name = "langchain_course_fewshot_selector"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

#### LengthBasedExampleSelector
The LengthBasedExampleSelector is useful when you're concerned about the length of the context window. It selects fewer examples for longer queries and more examples for shorter queries.

This method is effective for managing a large number of examples. It offers customization through various selectors, but it involves manual creation and selection of examples, which might not be ideal for every application type.

In [3]:
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [4]:
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]

example_template = """
Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_template
)

In [5]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25,
)

In [6]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n\n",
)

In [7]:
print(dynamic_prompt.format(input="big"))

Give the antonym of every input


Word: happy
Antonym: sad



Word: tall
Antonym: short



Word: energetic
Antonym: lethargic



Word: sunny
Antonym: gloomy


Word: big
Antonym:


#### SemanticSimilarityExampleSelector

The SemanticSimilarityExampleSelector for selecting examples based on their semantic resemblance to the input. This illustration showcases the process of creating an ExampleSelector, generating a prompt using a few-shot approach:

In [8]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import DeepLake
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Create a PromptTemplate
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Define some examples
examples = [
    {"input": "0°C", "output": "32°F"},
    {"input": "10°C", "output": "50°F"},
    {"input": "20°C", "output": "68°F"},
    {"input": "30°C", "output": "86°F"},
    {"input": "40°C", "output": "104°F"},
]

# create Deep Lake dataset
db = DeepLake(dataset_path=dataset_path)

# Embedding function
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Instantiate SemanticSimilarityExampleSelector using the examples
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, embeddings, db, k=1
)

# Create a FewShotPromptTemplate using the example_selector
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Convert the temperature from Celsius to Fahrenheit",
    suffix="Input: {temperature}\nOutput:", 
    input_variables=["temperature"],
)

# Test the similar_prompt with different inputs
print(similar_prompt.format(temperature="10°C"))   # Test with an input
print(similar_prompt.format(temperature="30°C"))  # Test with another input

# Add a new example to the SemanticSimilarityExampleSelector
similar_prompt.example_selector.add_example({"input": "50°C", "output": "122°F"})
print(similar_prompt.format(temperature="40°C")) # Test with a new input after adding the example

C:\ProgramData\Anaconda3b\envs\env_llm\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.4) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jfrancis/langchain_course_fewshot_selector


hub://jfrancis/langchain_course_fewshot_selector loaded successfully.


...

Convert the temperature from Celsius to Fahrenheit

Input: 30°C
Output: 86°F

Input: 30°C
Output:


Evaluating ingest: 0%|                                                                                   | 0/1 [00:00<?Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-k9LQtZKllzwGlzidOA94QWrY on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-k9LQtZKllzwGlzidOA94QWrY on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your 

Dataset(path='./deeplake/', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape     dtype  compression
  -------   -------   -------   -------  ------- 
 embedding  generic  (6, 1536)  float32   None   
    ids      text     (6, 1)      str     None   
 metadata    json     (6, 1)      str     None   
   text      text     (6, 1)      str     None   


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-k9LQtZKllzwGlzidOA94QWrY on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-k9LQtZKllzwGlzidOA94QWrY on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langcha

Convert the temperature from Celsius to Fahrenheit

Input: 40°C
Output: 104°F

Input: 40°C
Output:


##### 
Keep in mind that the SemanticSimilarityExampleSelector uses the Deep Lake vector store and OpenAIEmbeddings to measure semantic similarity. It stores the samples on the database in the cloud, and retrieves similar samples.

We created a PromptTemplate and defined several examples of temperature conversions. Next, we instantiated the SemanticSimilarityExampleSelector and created a FewShotPromptTemplate with the selector, example_prompt, and appropriate prefix and suffix.

Using SemanticSimilarityExampleSelector and FewShotPromptTemplate , we enabled the creation of versatile prompts tailored to specific tasks or domains, like temperature conversion in this case. These tools provide a customizable and adaptable solution for generating prompts that can be used with language models to achieve a wide range of tasks.

### 1. Output Parsers

There are three classes that we will introduce in this section. While the Pydrantic parser is the most powerful and flexible wrapper, knowing the other options for less complicated problems is beneficial. We will implement the thesaurus application in each section to better understand the details of each approach.

#### 1-1. PydanticOutputParser

This class instructs the model to generate its output in a JSON format and then extract the information from the response. You will be able to treat the parser’s output as a list, meaning it will be possible to index through the results without worrying about formatting.

In [9]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

# Define your desired data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")

    # Throw error in case of receiving a numbered-list from API
    @validator('words')
    def not_start_with_number(cls, field):
        for item in field:
            if item[0].isnumeric():
                raise ValueError("The word can not start with numbers!")
        return field

parser = PydanticOutputParser(pydantic_object=Suggestions)

We always import and follow the necessary libraries by creating the Suggestions schema class. There are two essential parts to this class:

    Expected Outputs: Each output is defined by declaring a variable with desired type, like a list of strings (: List[str]) in the sample code, or it could be a single string (: str) if you are expecting just one word/sentence as the response. Also, It is required to write a simple explanation using the Field function’s description attribute to help the model during inference. 
    Validators: It is possible to declare functions to validate the formatting. We ensure that the first character is not a number in the sample code. The function’s name is unimportant, but the @validator decorator must receive the same name as the variable you want to approve. (like @validator(’words’)) It is worth noting that the field variable inside the validator function will be a list if you specify it as one.

We will pass the created class to the PydanticOutputParser wrapper to make it a LangChain parser object. The next step is to prepare the prompt.

In [10]:
from langchain.prompts import PromptTemplate

template = """
Offer a list of suggestions to substitue the specified target_word based the presented context.
{format_instructions}
target_word={target_word}
context={context}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model_input = prompt.format_prompt(
            target_word="behaviour",
            context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."
)

The template variable is a string that can have named index placeholders using the following {variable_name} format. The template outlines our expectations for the model, including the expected formatting from the parser and the inputs. The PromptTemplate receives the template string with the details of each placeholder’s type. They could either be 1) input_variables whose value is initialized later on using the .format_prompt() function, or 2) partial_variables to be initialized instantly. 

In [11]:
from langchain.llms import OpenAI

# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
model = OpenAI(model_name='text-davinci-003', temperature=0.0)
# The temperature value could be anything between 0 to 1, where a higher number means the model is more creative. 
# Using larger value in production is a good practice if you deal with tasks requiring creative output.

output = model(model_input.to_string())

parser.parse(output)

Suggestions(words=['conduct', 'manner', 'action', 'demeanor', 'comportment'])

The parser object’s parse() function will convert the model’s string response to the format we specified. There is a list of words that you can index through and use in your applications.

##### Multiple Outputs Example

Here is a sample code for Pydantic class to process multiple outputs. It requests the model to suggest a list of words and present the reasoning behind each proposition.

Replace the template variable and Suggestion class with the following codes to run this example. The template changes will ask the model to present its reasoning, and the suggestion class declares a new output named reasons. Also, the validator function manipulates the output to ensure every reasoning ends with a dot. Another use case of the validator function could be output manipulation.

In [13]:
template = """
Offer a list of suggestions to substitute the specified target_word based on the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""

In [17]:
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")
    reasons: List[str] = Field(description="the reasoning of why this word fits the context")
    
    @validator('words')
    def not_start_with_number(cls, field):
        for item in field:
            if item[0].isnumeric():
                raise ValueError("The word can not start with numbers!")
        return field
    
    @validator('reasons')
    def end_with_dot(cls, field):
        for idx, item in enumerate( field ):
            if item[-1] != ".":
                field[idx] += "."
        return field
parser = PydanticOutputParser(pydantic_object=Suggestions)

prompt = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model_input = prompt.format_prompt(
            target_word="behaviour",
            context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."
)

In [18]:
output = model(model_input.to_string())

parser.parse(output)

Suggestions(words=['conduct', 'manner', 'demeanor', 'comportment'], reasons=['refers to the way someone acts in a particular situation.', 'refers to the way someone behaves in a particular situation.', 'refers to the way someone acts in a particular situation.', 'refers to the way someone behaves in a particular situation.'])

#### 1-2. CommaSeparatedOutputParser

It is evident from the name of this class that it manages comma-separated outputs. It handles one specific case: anytime you want to receive a list of outputs from the model. 

In [20]:
from langchain.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

The parser does not require a setting up step. Therefore it is less flexible. We can create the object by calling the class. The rest of the process for writing the prompt, initializing the model, and parsing the output is as follows.

In [21]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# Prepare the Prompt
template = """
Offer a list of suggestions to substitute the word '{target_word}' based the presented the following text: {context}.
{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model_input = prompt.format(
  target_word="behaviour",
  context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."
)

# Loading OpenAI API
model = OpenAI(model_name='text-davinci-003', temperature=0.0)

# Send the Request
output = model(model_input)
parser.parse(output)

['Conduct',
 'Actions',
 'Demeanor',
 'Mannerisms',
 'Attitude',
 'Performance',
 'Reactions',
 'Interactions',
 'Habits',
 'Repertoire',
 'Responses',
 'Interplay',
 'Interaction',
 'Intercourse',
 'Intercourse',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 'Interaction',
 '

The main difference compared to the previous subsection’s code is that we no longer need to call the .to_string() object since the prompt is already in string type.

#### 1-3. StructuredOutputParser

This is the first output parser implemented by the LangChain team. While it can process multiple outputs, it only supports texts and does not provide options for other data types, such as lists or integers. It can be used when you want to receive one response from the model. For example, only one substitute word in the thesaurus application.

In [22]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="words", description="A substitue word based on context"),
    ResponseSchema(name="reasons", description="the reasoning of why this word fits the context.")
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)

### 2. Fixing Errors

The parsers are powerful tools to dynamically extract the information from the prompt and validate it to some extent. Still, they do not guarantee a response. Imagine a situation where you deployed your application, and the model’s response [to a user’s request] is incomplete, causing the parser to throw an error. It is not ideal! In the following subsections, we will introduce two classes acting as fail-safe. They add a layer on top of the model’s response to help fix the errors.

#### 2-1. OutputFixingParser

This method tries to fix the parsing error by looking at the model’s response and the previous parser. It uses a Large Language Model (LLM) to solve the issue.

In [24]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Define your desired data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")
    reasons: List[str] = Field(description="the reasoning of why this word fits the context")

parser = PydanticOutputParser(pydantic_object=Suggestions)

missformatted_output = '{"words": ["conduct", "manner"], "reasoning": ["refers to the way someone acts in a particular situation.", "refers to the way someone behaves in a particular situation."]}'

parser.parse(missformatted_output)

OutputParserException: Failed to parse Suggestions from completion {"words": ["conduct", "manner"], "reasoning": ["refers to the way someone acts in a particular situation.", "refers to the way someone behaves in a particular situation."]}. Got: 1 validation error for Suggestions
reasons
  field required (type=value_error.missing)

As you can see in the error message, the parser correctly identified an error in our sample response (missformatted_output) since we used the word reasoning instead of the expected reasons key. The OutputFixingParser class could easily fix this error.

In [26]:
from langchain.llms import OpenAI
from langchain.output_parsers import OutputFixingParser

model = OpenAI(model_name='text-davinci-003', temperature=0.0)

outputfixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)
outputfixing_parser.parse(missformatted_output)

Suggestions(words=['conduct', 'manner'], reasons=['refers to the way someone acts in a particular situation.', 'refers to the way someone behaves in a particular situation.'])

The from_llm() function takes the old parser and a language model as input parameters. Then, It initializes a new parser for you that has the ability to fix output errors. In this case, it successfully identified the misnamed key and changed it to what we defined.

However, fixing the issues using this class is not always possible. Here is an example of using OutputFixingParser class to resolve an error with a missing key.

In [27]:
missformatted_output = '{"words": ["conduct", "manner"]}'

outputfixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

outputfixing_parser.parse(missformatted_output)

Suggestions(words=['conduct', 'manner'], reasons=["The word 'conduct' implies a certain behavior or action, while 'manner' implies a polite or respectful way of behaving."])

Looking at the output, it is evident that the model understood the key reasons missing from the response but didn’t have the context of the desired outcome. It created a list with one entry, while we expect one reason per word. This is why we sometimes need to use the RetryOutputParser class.

#### 2-2. RetryOutputParser

In some cases, the parser needs access to both the output and the prompt to process the full context. We first need to define the mentioned variables. The following codes initialize the LLM model, parser, and prompt, which were explained in more detail earlier.

In [29]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Define data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")
    reasons: List[str] = Field(description="the reasoning of why this word fits the context")

parser = PydanticOutputParser(pydantic_object=Suggestions)

# Define prompt
template = """
Offer a list of suggestions to substitue the specified target_word based the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model_input = prompt.format_prompt(target_word="behaviour", context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson.")

# Define Model
model = OpenAI(model_name='text-davinci-003', temperature=0.0)

Now, we can fix the same missformatted_output using the RetryWithErrorOutputParser class. It receives the old parser and a model to declare the new parser object, as we saw in the previous section. However, the parse_with_prompt function is responsible for fixing the parsing issue while requiring the output and the prompt.

In [30]:
from langchain.output_parsers import RetryWithErrorOutputParser

missformatted_output = '{"words": ["conduct", "manner"]}'

retry_parser = RetryWithErrorOutputParser.from_llm(parser=parser, llm=model)

retry_parser.parse_with_prompt(missformatted_output, model_input)

Suggestions(words=['conduct', 'manner'], reasons=["The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson, so 'conduct' is a suitable substitute.", "The students' behaviour was inappropriate, so 'manner' is a suitable substitute."])

The outputs show that the RetryOuputParser has the ability to fix the issue where the OuputFixingParser was not able to. The parser correctly guided the model to generate one reason for each word.

The best practice to incorporate these techniques in production is to catch the parsing error using a try: ... except: ... method. It means we can capture the errors in the except section and attempt to fix them using the mentioned classes. It will limit the number of API calls and avoid unnecessary costs that are associated with it.